<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Constants" data-toc-modified-id="Constants-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Constants</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Functions</a></span></li><li><span><a href="#Run-it" data-toc-modified-id="Run-it-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run it</a></span></li><li><span><a href="#End-of-notebook" data-toc-modified-id="End-of-notebook-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>End of notebook</a></span></li></ul></div>

# Setup

In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
import numpy as np
import pandas as pd
from requests import HTTPError
import pvlib
import pytz
import folium
import geopy.distance
from dotenv import load_dotenv
import os

In [8]:
#%config InlineBackend.figure_format = 'retina'

In [9]:
load_dotenv()
NREL_API_KEY = os.getenv("NREL_API_KEY")
NREL_API_EMAIL = os.getenv("NREL_API_EMAIL")

# Constants

In [12]:
PV_PANEL_MODEL = pvlib.pvsystem.retrieve_sam("SandiaMod")[
    "Canadian_Solar_CS5P_220M___2009_"
]
INVERTER_MODEL = pvlib.pvsystem.retrieve_sam("cecinverter")[
    "ABB__MICRO_0_25_I_OUTD_US_208__208V_"
]

LAT_SAN_FRANCISCO = 37.78128901022419
LON_SAN_FRANCISCO = -122.4589148156449
LAT_NORTH_LIMIT = 55
LON_WEST_LIMIT = -131
LAT_SOUTH_LIMIT = 15
LON_EAST_LIMIT = -69

# default pvlib.iotools.psm3.ATTRIBUTES plus four more
SOLAR_WEATHER_ATTRIBUTES = (
    "air_temperature",
    "dew_point",
    "dhi",
    "clearsky_dhi",
    "dni",
    "clearsky_dni",
    "ghi",
    "clearsky_ghi",
    "surface_albedo",
    "surface_pressure",
    "wind_direction",
    "wind_speed",
)

# Functions

In [21]:
# This function taken from week 4 assigment
# https://app.hex.tech/d98cf7a0-1d0f-4e8d-82bd-7911e8769831/hex/808fbd38-fb0d-411b-b5cd-8bf6ec41144d/draft/logic?selectedStaticCellId=653a526e-bf1a-4faf-ac1d-85c5df858d7d
# We're using `pvlib` to simulate how much electricity our PV system would generate given historical "solar weather" data

def simulate_pv_ouptput(
    solar_weather_timeseries,
    latitude,
    longitude,
    altitude,
    pv_array_tilt,
    pv_array_azimuth,
    pv_panel_model,
    inverter_model,
):
    # Adapted from example: https://pvlib-python.readthedocs.io/en/v0.9.0/introtutorial.html?highlight=total_irradiance#procedural
    # new link: https://pvlib-python.readthedocs.io/en/v0.11.1/user_guide/introtutorial.html

    # First, we model the position of the sun relative to our chosen location over the simulation year
    solar_position_timeseries = pvlib.solarposition.get_solarposition(
        time=solar_weather_timeseries.index,
        latitude=latitude,
        longitude=longitude,
        altitude=altitude,
        temperature=solar_weather_timeseries["temp_air"],
    )

    # We combine solar position with historical solar weather data to model total irradiance for our PV panel
    total_irradiance_timeseries = pvlib.irradiance.get_total_irradiance(
        pv_array_tilt,
        pv_array_azimuth,
        solar_position_timeseries["apparent_zenith"],
        solar_position_timeseries["azimuth"],
        solar_weather_timeseries["dni"],
        solar_weather_timeseries["ghi"],
        solar_weather_timeseries["dhi"],
        dni_extra=pvlib.irradiance.get_extra_radiation(solar_weather_timeseries.index),
        model="haydavies",
    )

    # We then model air mass & angle of incidence, which we combine with total irradiance to model "effective" irradiance on our PV panel
    # Air mass is a measure of the path length of solar radiation through the atmosphere
    absolute_airmass_timeseries = pvlib.atmosphere.get_absolute_airmass(
        pvlib.atmosphere.get_relative_airmass(
            solar_position_timeseries["apparent_zenith"]
        ),
        pvlib.atmosphere.alt2pres(altitude),
    )

    # Angle of incidence is the angle of the sun's rays relative to the panel's surface
    angle_of_incidence_timeseries = pvlib.irradiance.aoi(
        pv_array_tilt,
        pv_array_azimuth,
        solar_position_timeseries["apparent_zenith"],
        solar_position_timeseries["azimuth"],
    )

    # This is where we combine the direct and diffuse irradiance, taking into account the air mass that the sunlight has to travel through
    effective_irradiance_timeseries = pvlib.pvsystem.sapm_effective_irradiance(
        total_irradiance_timeseries["poa_direct"],
        total_irradiance_timeseries["poa_diffuse"],
        absolute_airmass_timeseries,
        angle_of_incidence_timeseries,
        pv_panel_model,
    )

    # We model the temperature within the PV panel ("cell temperature"), which affects the efficiency of the panels
    cell_temperature_timeseries = pvlib.temperature.sapm_cell(
        total_irradiance_timeseries["poa_global"],
        solar_weather_timeseries["temp_air"],
        solar_weather_timeseries["wind_speed"],
        **pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS["sapm"]["open_rack_glass_glass"],
    )

    # Finally we put it all together:

    # We simulate the DC electricity output of our PV panel given the effective solar irradiance and cell temperature)
    dc_electricity_timeseries = pvlib.pvsystem.sapm(
        effective_irradiance_timeseries, 
        cell_temperature_timeseries, 
        pv_panel_model
    )

    # And then we simulate the inverter converting the DC output into AC output
    ac_electricity_timeseries_watts = pvlib.inverter.sandia(
        dc_electricity_timeseries["v_mp"], 
        dc_electricity_timeseries["p_mp"], 
        inverter_model
    )

    # Wrap the results all up into a dataframe for plotting!
    pv_model_results = pd.DataFrame(
        {
            "PV Array Output (Wh)": dc_electricity_timeseries["i_mp"] * dc_electricity_timeseries["v_mp"],
            "Inverter Output (Wh)": ac_electricity_timeseries_watts,
            "Solar azimuth (°)": solar_position_timeseries["azimuth"],
            "Solar elevation (°)": solar_position_timeseries["apparent_elevation"],
        }
    )
    pv_model_results["timestamp"] = pv_model_results.index.map(
        lambda utc_time: utc_time.astimezone(pytz.timezone('UTC'))
    )
    return pv_model_results

In [22]:
def solar_potential(lat: float, lon: float) -> float:
    """
    Take parameters specifying a real or simulated location. Return its yearly solar potential.

    Args:
        lat (float): Latitude
        lon (float): Longitude
        [additional parameters to be implemented]

    Returns:
        generation (float): kWh/year/m2 of an optimally positioned solar panel
    """

    try:
        solar_weather_timeseries, solar_weather_metadata = pvlib.iotools.get_psm3(
            latitude=lat,
            longitude=lon,
            names=2019,
            leap_day=False,
            attributes=SOLAR_WEATHER_ATTRIBUTES,
            map_variables=True,
            api_key=NREL_API_KEY,
            email=NREL_API_EMAIL,
        )
    except HTTPError:
        print(f"Missing solar weather for {lat}, {lon}")
        return None
    except:
        print(f"Other error for {lat}, {lon}")
        return None
    #warnings.warn("All altitudes are set to 0.")
    pv_model_results = simulate_pv_ouptput(
        solar_weather_timeseries,
        latitude=lat,
        longitude=lon,
        altitude=0,
        pv_array_tilt=lat,
        pv_array_azimuth=180,
        pv_panel_model=PV_PANEL_MODEL,
        inverter_model=INVERTER_MODEL,
    )
    generation_kWh_year = pv_model_results["Inverter Output (Wh)"].sum() / 1000
    generation_kWh_year_m2 = generation_kWh_year / PV_PANEL_MODEL.Area
    return generation_kWh_year_m2

# Run it

In [23]:
solar_weather_timeseries, solar_weather_metadata = pvlib.iotools.get_psm3(
    latitude=LAT_SAN_FRANCISCO,
    longitude=LON_SAN_FRANCISCO,
    names=2019,
    leap_day=False,
    attributes=SOLAR_WEATHER_ATTRIBUTES,
    map_variables=True,
    api_key=NREL_API_KEY,
    email=NREL_API_EMAIL,
)

In [24]:
solar_weather_timeseries.head(24)

,Year,Month,Day,Hour,Minute,temp_air,temp_dew,dhi,dhi_clear,dni,dni_clear,ghi,ghi_clear,albedo,pressure,wind_direction,wind_speed
2019-01-01 00:30:00-08:00,2019,1,1,0,30,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1015.0,26.0,3.9
2019-01-01 01:30:00-08:00,2019,1,1,1,30,4.4,-0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1016.0,26.0,3.9
2019-01-01 02:30:00-08:00,2019,1,1,2,30,4.3,-0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1017.0,27.0,3.9
2019-01-01 03:30:00-08:00,2019,1,1,3,30,4.1,-1.3,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1017.0,28.0,3.8
2019-01-01 04:30:00-08:00,2019,1,1,4,30,3.8,-1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1017.0,27.0,3.6
2019-01-01 05:30:00-08:00,2019,1,1,5,30,3.6,-1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1018.0,26.0,3.4
2019-01-01 06:30:00-08:00,2019,1,1,6,30,3.4,-1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.10,1019.0,29.0,3.2
2019-01-01 07:30:00-08:00,2019,1,1,7,30,4.2,-1.7,10.0,10.0,153.0,153.0,11.0,11.0,0.10,1020.0,33.0,3.2
2019-01-01 08:30:00-08:00,2019,1,1,8,30,6.0,-1.3,38.0,34.0,641.0,698.0,148.0,153.0,0.10,1021.0,37.0,3.4
2019-01-01 09:30:00-08:00,2019,1,1,9,30,7.8,-2.1,47.0,47.0,851.0,851.0,313.0,313.0,0.10,1021.0,40.0,3.2


In [25]:
# This json of congressional district boundries comes from
# https://eric.clst.org/tech/usgeojson/
with open("gz_2010_us_500_11_20m.json", "r") as f:
    congressional_json = json.load(f)

In [26]:
congressional_df = pd.DataFrame()
for i, feature in enumerate(congressional_json["features"]):
    if i % 50 == 0:
        print(f"i = {i}")
    if i >= 50:
        break
    property_dict = feature["properties"]
    property_dict["json_index"] = i
    property_row = pd.Series(property_dict)
    coordinates_list = feature["geometry"]["coordinates"]
    if len(coordinates_list) == 1:
        coordinates_array = np.array(coordinates_list[0])
    else:
        coordinates_list_of_lists = []
        for sub_list in coordinates_list:
            if len(sub_list) == 1 or i == 83:
                # CD 83 has another layer, just throwing aways the second polygon
                # because I can't be bothered
                # One can imagine a better solution than this hack
                sub_list = sub_list[0]
            else:
                # The occasional case when there's one fewer dimension.
                pass
            coordinates_list_of_lists += [sub_list]
        coordinates_array = np.concatenate(coordinates_list_of_lists)
    full_row_list = []
    for coord in coordinates_array:
        full_row = pd.concat(
            [property_row, pd.Series({"lat": coord[1], "lon": coord[0]})]
        )
        full_row_list += [full_row]
    congressional_df = pd.concat(
        [congressional_df] + [r.to_frame().T for r in full_row_list]
    )

i = 0
i = 50


In [27]:
congressional_summary_df = (
    congressional_df.groupby(
        ["GEO_ID", "STATE", "CD", "NAME", "LSAD", "CENSUSAREA", "json_index"]
    )
    .agg({"lat": "mean", "lon": "mean", "GEO_ID": "count"})
    .sort_values("json_index")
)
congressional_summary_df.head()

,,,,,,,lat,lon,GEO_ID
GEO_ID,STATE,CD,NAME,LSAD,CENSUSAREA,json_index,,,
5001100US0101,01,01,1,CD,6304.328,0,30.780483,-87.816111,280
5001100US0401,04,01,1,CD,58573.066,1,35.68314,-111.510317,464
5001100US0402,04,02,2,CD,20222.310,2,35.894638,-112.356198,472
5001100US0403,04,03,3,CD,598.057,3,33.801145,-112.059257,12
5001100US0404,04,04,4,CD,198.996,4,33.408975,-112.131399,7


In [ ]:
generation_list = []
for i, row in congressional_summary_df.iterrows():
    point = (row.lat, row.lon)
    if i[6] % 50 == 0:
        print(f"i[6] = {i[6]}")
        print(f"{point}")
    generation_list += [(point[0], point[1], solar_potential(point[0], point[1]), i[0])]

i[6] = 0
(30.780483174999986, -87.81611115357144)


In [30]:
generation_df = pd.DataFrame(generation_list, columns=["lat", "lon", "generation", "GEO_ID"]).dropna().reset_index()

In [31]:
generation_df.head()

,index,lat,lon,generation,GEO_ID
0,0,30.780483,-87.816111,199.246894,5001100US0101
1,1,35.683140,-111.510317,257.624490,5001100US0401
2,2,35.894638,-112.356198,260.533429,5001100US0402
3,3,33.801145,-112.059257,249.919691,5001100US0403
4,4,33.408975,-112.131399,247.056649,5001100US0404


In [32]:
generation_df.to_csv("generation_2024_11_15_01.csv")

In [33]:
ncolors = 20
palette = sns.color_palette("flare_r", ncolors)
hex_colors = palette.as_hex()
vmin = generation_df.generation.min()
vmax = generation_df.generation.max()
vrange = vmax - vmin
generation_df["color"] = generation_df.generation.apply(
    lambda x: hex_colors[int((ncolors - 1) * (x - vmin) / vrange)]
)

In [34]:
m = folium.Map(
    location=(generation_df.lat.mean(), generation_df.lon.mean()),
    zoom_start=4,
    height=800,
    width=1100,
)
for i, row in generation_df.iterrows():
    folium.CircleMarker(
        location=[row.lat, row.lon],
        fill=True,
        stroke=False,
        fill_opacity=0.8,
        radius=6,
        color=row.color,
    ).add_to(m)
m

In [35]:
json_output = {"type": "FeatureCollection", "features": []}
for i, row in congressional_summary_df.iterrows():
    g = generation_df[generation_df.GEO_ID == i[0]]
    if g.shape[0] == 0:
        print(f"passing on GEO_ID {i[0]}")
        continue
    assert g.shape[0] == 1
    assert g.lat.values[0] == row.lat
    assert g.lon.values[0] == row.lon
    this_feature = {
        "type": "Feature",
        "properties": {
            "GEO_ID": i[0],
            "STATE": i[1],
            "CD": i[2],
            "NAME": i[3],
            "LSAD": i[4],
            "CENSUSAREA": i[5],
            "generation_real": g.generation.values[0],
        },
        "geometry": {"type": "Point", "coordinates": [row.lon, row.lat]},
    }
    json_output["features"] += [this_feature]

In [36]:
with open('real_generation_points.json', 'w') as f:
    json.dump(json_output, f)

# End of notebook